---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

# - LOAD & CLEAN data.

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

## 1) Know each table a little bit more to know how to go properly.

In [31]:
import pandas as pd
train_df = pd.read_csv('data/train.csv',encoding='latin1')

print(len(train_df),'\n')
print(train_df.columns)
train_df.head(3)

C:\Users\Assem Salama\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (11,12,31) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


250306 

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'payment_amount', 'balance_due',
       'payment_date', 'payment_status', 'collection_status',
       'grafitti_status', 'compliance_detail', 'compliance'],
      dtype='object')


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [32]:
test_df = pd.read_csv('data/test.csv')

print(len(test_df),'\n')
print(test_df.columns)
test_df.head(3)

61001 

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'grafitti_status'],
      dtype='object')


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
0,284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,...,Failure to secure City or Private solid waste ...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
1,285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN
2,285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Improper placement of Courville container betw...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN


In [33]:
address = pd.read_csv('data/addresses.csv')
latlons = pd.read_csv('data/latlons.csv')

print(len(address))
print(len(latlons))

311307
121769


_____

## 2) Join `address` & `latlos` with respect to address because it has the larger num of instances. Then, you face a NANs that need to drop. Finally , reset the index.


NOTE: to join you need to make the two tables has the same same index to join accordingly. 

In [34]:
address_lalo = address.set_index('address').join(latlons.set_index('address'),how='left').dropna().reset_index()
print(len(address_lalo))
address_lalo.head(3)

311299


,address,ticket_id,lat,lon
0,"-11064 gratiot, Detroit MI",328722,42.406935,-82.995599
1,"-11871 wilfred, Detroit MI",350971,42.411288,-82.993674
2,"-15126 harper, Detroit MI",344821,42.406402,-82.957525


### 3) we'll merge the `train_df` &  `address_lalo` on `ticket_id` which means ticket id that exists in both dataframe.

In [35]:
train_data = pd.merge(train_df , address_lalo , on='ticket_id').set_index('ticket_id')
print(len(train_data))
train_data.head(3)

250303


,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,...,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance,address,lat,lon
ticket_id,,,,,,,,,,,,,,,,,,,,,
22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,IL,...,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,"2900 tyler, Detroit MI",42.390729,-83.124268
27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,MI,...,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0,"4311 central, Detroit MI",42.326937,-83.135118
22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,MI,...,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,"1449 longfellow, Detroit MI",42.380516,-83.096069


### 4) we'll do the same procedures as above.But withrespect to the `test_df` where it's our benchmark.This is gonna bring a NANs that need to be filled with the last valid observation.

In [36]:
test_data = pd.merge(test_df , address_lalo ,how = 'left' ,on='ticket_id').set_index('ticket_id')
print(len(test_data))
test_data.head(3)

61001


,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,...,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,address,lat,lon
ticket_id,,,,,,,,,,,,,,,,,,,,,
284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,MI,...,20.0,10.0,20.0,0.0,0.0,250.0,NaN,"10041 roseberry, Detroit MI",42.407581,-82.986642
285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,MI,...,20.0,10.0,100.0,0.0,0.0,1130.0,NaN,"18520 evergreen, Detroit MI",42.426239,-83.238259
285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,MI,...,20.0,10.0,10.0,0.0,0.0,140.0,NaN,"18520 evergreen, Detroit MI",42.426239,-83.238259


In [37]:
# because of left join we got a NAN values.
# SO, we need to fill these NAN with the last valid observation
# using fillna(method  = 'pad or ffill')
test_data['lat'].isnull().unique()


array([False,  True])

`Method`: to use for filling holes in reindexed Series.
- pad / ffill: propagate last valid observation forward to next valid 
- backfill / bfill: use next valid observation to fill gap.

In [38]:
test_data['lat'].fillna(method='pad' , inplace = True)
test_data['lon'].fillna(method='pad' , inplace = True)

test_data['lat'].isnull().unique()

array([False])

### 5) Remove `not responsible` compliance instances. Keep only True or False records from `train_data`. Make the `compliance` column as [int] to ease getting the probability.

In [39]:
train_data = train_data[(train_data['compliance'] == 0) | (train_data['compliance'] == 1)]
train_data.compliance.unique()

array([0., 1.])

In [40]:
train_data['compliance']= train_data['compliance'].astype(int)

# 6) CALULATE TIME gap.
- between the hearing time and the time when the issue revealed.

The strptime() method creates a datetime object from the given string.
Note: You cannot create datetime object from every string. The string needs to be in a certain format.

In [41]:
# create the function to do so.
from datetime import datetime
def time_gap(hearing_date_str , ticket_issued_date_str):
    if not hearing_date_str or type(hearing_date_str)!=str: return 73

    hearing_date = datetime.strptime(hearing_date_str,'%Y-%m-%d %H:%M:%S')
    ticket_issued_date = datetime.strptime(ticket_issued_date_str,'%Y-%m-%d %H:%M:%S')
    gap = hearing_date - ticket_issued_date
    diff = gap.days
    return diff

In [42]:
# create the gap_time column in each data set.
train_data['time_gap'] = train_data.apply(lambda x: time_gap(x['hearing_date'] ,
                                                             x['ticket_issued_date']) ,axis =1)
test_data['time_gap'] = test_data.apply(lambda x: time_gap(x['hearing_date'] ,
                                                           x['ticket_issued_date']) ,axis =1)

In [43]:
# check.
train_data[['time_gap','hearing_date', 'ticket_issued_date']].head(3)

,time_gap,hearing_date,ticket_issued_date
ticket_id,,,
22056,369,2005-03-21 10:30:00,2004-03-16 11:40:00
27586,378,2005-05-06 13:30:00,2004-04-23 12:30:00
22046,323,2005-03-21 10:30:00,2004-05-01 11:50:00


# 7) Select the `features` i'll use in model' training phase, and which i won't.See if any of the features need altering its d-type.

In [44]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159878 entries, 22056 to 284333
Data columns (total 37 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   agency_name                 159878 non-null  object 
 1   inspector_name              159878 non-null  object 
 2   violator_name               159852 non-null  object 
 3   violation_street_number     159878 non-null  float64
 4   violation_street_name       159878 non-null  object 
 5   violation_zip_code          0 non-null       float64
 6   mailing_address_str_number  157320 non-null  float64
 7   mailing_address_str_name    159875 non-null  object 
 8   city                        159878 non-null  object 
 9   state                       159794 non-null  object 
 10  zip_code                    159877 non-null  object 
 11  non_us_str_code             3 non-null       object 
 12  country                     159878 non-null  object 
 13  ticket_iss

In [45]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61001 entries, 284932 to 369851
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   agency_name                 61001 non-null  object 
 1   inspector_name              61001 non-null  object 
 2   violator_name               60973 non-null  object 
 3   violation_street_number     61001 non-null  float64
 4   violation_street_name       61001 non-null  object 
 5   violation_zip_code          24024 non-null  object 
 6   mailing_address_str_number  59987 non-null  object 
 7   mailing_address_str_name    60998 non-null  object 
 8   city                        61000 non-null  object 
 9   state                       60670 non-null  object 
 10  zip_code                    60998 non-null  object 
 11  non_us_str_code             0 non-null      float64
 12  country                     61001 non-null  object 
 13  ticket_issued_date       

In [46]:
# choosen features.
choosen_feature = ['agency_name',
        'violation_street_name',
        'state',
        'violation_code',
        'late_fee',
        'fine_amount',
        'discount_amount',
        'judgment_amount',
        'lat',
        'lon',
        'disposition',
        'time_gap']

# features need to be removed because of the `data leakage`.
deleted_train_only_columns = ['payment_amount', 'payment_date', 'payment_status', 'balance_due',
        'collection_status', 'compliance_detail']
train_data.drop(deleted_train_only_columns,axis=1,inplace=True)

# assign the appropriate d-type required.
for df in [train_data,test_data]:
    df['agency_name'] = df['agency_name'].astype('category')
    df['violation_street_name'] = df['violation_street_name'].astype('category')
    df['state'] = df['state'].astype('category')
    df['violation_code'] = df['violation_code'].astype('category')
    df['disposition'] = df['disposition'].astype('category')

# 8)  Guide to Encoding Categorical Values.

## Introduction
- In many practical Data Science activities, the data set will contain categorical variables. These variables are typically stored as text values which represent various traits. Some examples include color (“Red”, “Yellow”, “Blue”), size (“Small”, “Medium”, “Large”) or geographic designations (State or Country). Regardless of what the value is used for, the challenge is determining how to use this data in the analysis. Many machine learning algorithms can support categorical values without further manipulation but there are many more algorithms that do not. Therefore, the analyst is faced with the challenge of figuring out how to turn these text attributes into numerical values for further processing.


- As with many other aspects of the Data Science world, there is no single answer on how to approach this problem. Each approach has trade-offs and has potential impact on the outcome of the analysis. Fortunately, the python tools of pandas and scikit-learn provide several approaches that can be applied to transform the categorical data into suitable numeric values. 

In [47]:
# Convert cetegory columns to integers
categorical_columns = train_data.select_dtypes(['category']).columns
for df in [train_data,test_data]:
    df[categorical_columns] = df[categorical_columns].apply(lambda x: x.cat.codes)

# 9) preparing data to the ML algorithm and applying it.

In [48]:
final_train = train_data[choosen_feature]
final_test = test_data[choosen_feature]
target_value = train_data['compliance']

In [49]:
X = final_train
y = target_value

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 0)
# train the Model
GBclf = GradientBoostingClassifier(learning_rate=0.1,max_depth= 10 ,random_state = 0).fit(X_train,y_train)

In [50]:
# get y_scores
yscores_GBclf = GBclf.decision_function(X_test)

# get false positive rate , True positive rate.
fpr_GBclf, tpr_GBclf, _ = roc_curve(y_test ,yscores_GBclf)
# get the area under curve.
roc_auc_GBclf = auc(fpr_GBclf , tpr_GBclf)
# get the accuracy.
accuracy_GBclf = GBclf.score(X_test, y_test)

print("accuracy = {:.4f}   AUC = {:.4f}".format(accuracy_GBclf,  roc_auc_GBclf))

accuracy = 0.9420   AUC = 0.8280


In [51]:
y_probability = GBclf.predict_proba(final_test)[:,1]
y_probability

array([0.1474124 , 0.01899973, 0.07991094, ..., 0.0586243 , 0.0586243 ,
       0.7953502 ])

In [52]:
final_test['compliance'] = y_probability
final_test['compliance']

<ipython-input-52-509580581656>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_test['compliance'] = y_probability


ticket_id
284932    0.147412
285362    0.019000
285361    0.079911
285338    0.334474
285346    0.468640
            ...   
376496    0.020403
376497    0.020403
376499    0.058624
376500    0.058624
369851    0.795350
Name: compliance, Length: 61001, dtype: float64

In [24]:
import pandas as pd
import numpy as np

def blight_model():
    import pandas as pd
    import numpy as np
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.svm import SVC
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_curve, auc
    from sklearn.metrics import roc_auc_score
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.model_selection import GridSearchCV
    
    train_df = pd.read_csv('data/train.csv',encoding='latin1')
    test_df = pd.read_csv('data/test.csv')
    address = pd.read_csv('data/addresses.csv')
    latlons = pd.read_csv('data/latlons.csv')

    
    address_lalo = address.set_index('address').join(latlons.set_index('address'),how='left').dropna().reset_index()
    train_data = pd.merge(train_df , address_lalo , on='ticket_id').set_index('ticket_id')
    test_data = pd.merge(test_df , address_lalo ,how = 'left' ,on='ticket_id').set_index('ticket_id')
    
    test_data['lat'].fillna(method='pad' , inplace = True)
    test_data['lon'].fillna(method='pad' , inplace = True)
    
    train_data = train_data[(train_data['compliance'] == 0) | (train_data['compliance'] == 1)]
    train_data['compliance']= train_data['compliance'].astype(int)
    
    from datetime import datetime
    def time_gap(hearing_date_str , ticket_issued_date_str):
        
        if not hearing_date_str or type(hearing_date_str)!=str: return 73

        hearing_date = datetime.strptime(hearing_date_str,'%Y-%m-%d %H:%M:%S')
        ticket_issued_date = datetime.strptime(ticket_issued_date_str,'%Y-%m-%d %H:%M:%S')
        gap = hearing_date - ticket_issued_date
        diff = gap.days
        return diff
    
    train_data['time_gap'] = train_data.apply(lambda x: time_gap(x['hearing_date'] ,
                                                             x['ticket_issued_date']) ,axis =1)
    test_data['time_gap'] = test_data.apply(lambda x: time_gap(x['hearing_date'] ,
                                                           x['ticket_issued_date']) ,axis =1)
    
    
    
    # choosen features.
    choosen_feature = ['agency_name',
        'violation_street_name',
        'state',
        'violation_code',
        'late_fee',
        'fine_amount',
        'discount_amount',
        'judgment_amount',
        'lat',
        'lon',
        'time_gap']

    # features need to be removed because of the `data leakage`.
    deleted_train_only_columns = ['payment_amount', 'payment_date', 'payment_status', 'balance_due',
            'collection_status', 'compliance_detail']
    train_data.drop(deleted_train_only_columns,axis=1,inplace=True)

    # assign the appropriate d-type required.
    for df in [train_data,test_data]:
        df['agency_name'] = df['agency_name'].astype('category')
        df['violation_street_name'] = df['violation_street_name'].astype('category')
        df['state'] = df['state'].astype('category')
        df['violation_code'] = df['violation_code'].astype('category')
        df['disposition'] = df['disposition'].astype('category')
    
    # Convert cetegory columns to integers
    categorical_columns = train_data.select_dtypes(['category']).columns
    for df in [train_data,test_data]:
        df[categorical_columns] = df[categorical_columns].apply(lambda x: x.cat.codes)
        
        
    final_train = train_data[choosen_feature]
    final_test = test_data[choosen_feature]
    target_value = train_data['compliance']    
    
    X = final_train
    y = target_value

    X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 0)
    # train the Model
    GBclf = GradientBoostingClassifier(learning_rate=0.1,max_depth= 5 ,random_state = 0).fit(X_train,y_train)
    # get y_scores
    yscores_GBclf = GBclf.decision_function(X_test)
    # get false positive rate , True positive rate.
    fpr_GBclf, tpr_GBclf, _ = roc_curve(y_test ,yscores_GBclf)
    # get the area under curve.
    roc_auc_GBclf = auc(fpr_GBclf , tpr_GBclf)
    # get the accuracy.
    accuracy_GBclf = GBclf.score(X_test, y_test)
    print("accuracy = {:.4f}   AUC = {:.4f}".format(accuracy_GBclf,  roc_auc_GBclf))

    y_probability = GBclf.predict_proba(final_test)[:,1]
    final_test['compliance'] = y_probability

    
    return final_test['compliance']

In [25]:
blight_model()

C:\Users\Assem Salama\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (11,12,31) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


accuracy = 0.9346   AUC = 0.7943


<ipython-input-24-e42a6abf946f>:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_test['compliance'] = y_probability


ticket_id
284932    0.085164
285362    0.017509
285361    0.071615
285338    0.081694
285346    0.093148
            ...   
376496    0.018970
376497    0.018970
376499    0.071046
376500    0.071046
369851    0.294378
Name: compliance, Length: 61001, dtype: float64